In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

Input Data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/sales_predictions.csv')

In [5]:
#dropping empty data
df.dropna(subset = ['Item_Weight'], inplace=False)
#fix inconsistencies
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('LF', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('reg', 'Regular')
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('low fat', 'Low Fat')

In [10]:
#make copy
og_df = df
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.  

In [7]:
y = df['Item_Outlet_Sales']
x = df.drop(columns = 'Item_Outlet_Sales')

Perform a train test split 

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

Create a preprocessing object to prepare the dataset for Machine Learning

In [33]:
df['Item_Fat_Content'].value_counts()
train_ohe = df['Item_Fat_Content'].replace({'Low Fat':0, 'Regular':1}, inplace=True)

In [32]:
df['Item_Type'].value_counts()
train_ohe = df['Item_Type'].replace({'Fruits and Vegetables':0, 'Snack Foods':1, 'Household': 3, 'Frozen Foods': 4, 'Dairy': 5, 'Canned': 6, 'Baking Goods': 7, 'Health and Hygiene': 8,
                                    'Soft Drinks': 9, 'Meat': 10, 'Breads': 11, 'Hard Drinks': 12, 'Others': 13, 'Starchy Foods': 14, 'Breakfast':15, 'Seafood':16}, inplace=True)

In [17]:
new = df['Outlet_Size'].mode()
df['Outlet_Size'].fillna(new, inplace = True)

In [31]:
df['Outlet_Size'].value_counts()
train_ohe = df['Outlet_Size'].replace({'Small':0, 'Medium':1, 'High': 2}, inplace=True)

In [35]:
df['Outlet_Location_Type'].value_counts()
train_ohe = df['Outlet_Location_Type'].replace({'Tier 1':0, 'Tier 2':1, 'Tier 3': 2}, inplace=True)

In [34]:
df['Outlet_Type'].value_counts()
train_ohe = df['Outlet_Type'].replace({'Grocery Store':0, 'Supermarket Type1':1, 'Supermarket Type2': 2, 'Supermarket Type3': 3}, inplace=True)

In [30]:
region_selector = make_column_selector(dtype_include='object')
train_region_data = x_train[region_selector(x_train)]
test_region_data = x_test[region_selector(x_test)]
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_encoder.fit(train_region_data)
train_ohe = ohe_encoder.transform(train_region_data)
test_ohe = ohe_encoder.transform(test_region_data)
ohe_column_names = ohe_encoder.get_feature_names_out(train_region_data.columns)
train_ohe = pd.DataFrame(train_ohe, columns=ohe_column_names)
test_ohe = pd.DataFrame(test_ohe, columns=ohe_column_names)
train_ohe

,Item_Identifier_DRA12,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,Item_Identifier_DRB48,Item_Identifier_DRC01,Item_Identifier_DRC12,...,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Size_nan,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
